In [ ]:
# HPC pass/fail binary classification model by Jiwon Park
# version 1.2 (2023. 06. 04)

In [ ]:
import pandas as pd
import seaborn as sb
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime
import os
import sys
from sklearn.base import BaseEstimator
from sklearn.metrics import hamming_loss
import eli5
from eli5.sklearn import PermutationImportance

# customize your project folder
rawdata = pd.read_csv('Final data 230125.csv', sep=',')  #Load your CSV data 

In [ ]:
from sklearn.inspection import permutation_importance

In [ ]:
print("True labels are")
print(rawdata["HPC_fail"].value_counts())

In [ ]:
# input presets
# LNA and HNA are subpopulation of intact cell count (ICC)
input1=rawdata.loc[:, ['Free_Cl', 'ATP', 'LNA','HNA','HPC_fail']] #including all data
input2=rawdata.loc[:, ['ATP', 'LNA','HNA','HPC_fail']]            #excepting free chlorine
input3=rawdata.loc[:, ['Free_Cl', 'LNA','HNA','HPC_fail']]        #excepting ATP
input4=rawdata.loc[:, ['Free_Cl', 'ATP', 'ICC','HPC_fail']]       #replacing HNA/LNA to ICC
input5=rawdata.loc[:, ['Free_Cl', 'ATP','HPC_fail']]              #excepting all FCM data 
input6=rawdata.loc[:, ['Free_Cl', 'ICC','HPC_fail']]              #excepting ATP, but replacing HNA/LNA to ICC
input7=rawdata.loc[:, ['LNA','HNA','HPC_fail']]                   #only HNA/LNA
input8=rawdata.loc[:, ['ATP','HPC_fail']]                         #only ATP
input9=rawdata.loc[:, ['ICC','HPC_fail']]                         #only ICC
input10=rawdata.loc[:, ['Free_Cl','HPC_fail']]                     #only free chlorine

In [ ]:
### user control panel ###

# input series selection
table=input1

# activation function selection (for all hidden layers)
ActFunc="relu" #"relu" or "tanh"

nodes1="8" #number of 1st hidden nodes

In [ ]:
table_norm = (table - table.min()) / (table.max()-table.min())
table_shuffle = table_norm.sample(frac=1)  # Sample order shuffle
print(table_shuffle.head()) # check your normalized data 
table_np = table_shuffle.to_numpy() 

In [ ]:
train_idx = int(len(table_np)*0.7) 
train_X, train_Y = table_np[:train_idx, :-1], table_np[:train_idx, -1]
test_X, test_Y = table_np[train_idx:, :-1], table_np[train_idx:, -1]
train_Y = tf.keras.utils.to_categorical(train_Y, num_classes = 2) 
test_Y = tf.keras.utils.to_categorical(test_Y, num_classes = 2)

In [ ]:
# activation function option 
# Too much modes or dense layers result in overfitting of model
# I recommend using 'relu' in hidden layer activation function
model =tf.keras.Sequential([
    tf.keras.layers.Dense(units=nodes1, activation=ActFunc, input_shape=(len(table.columns)-1,)), 
    tf.keras.layers.Dense(units=8, activation=ActFunc), 
    tf.keras.layers.Dense(units=2, activation='softmax')
])

# optimizers: Adam, SGD, Adagrad, Nadam, or else
# You can optimize learning rate
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.2), 
              loss='binary_crossentropy', metrics=['accuracy']) 

#optimization is needed for numbers in epochs and batch size
#validation split affects the size of training/test data (we recommend 0.2 or 0.3)
history = model.fit(train_X, train_Y, epochs=500, batch_size=50, validation_split=0.3,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=50, monitor="val_loss")]) 
#Earlystopping will cease epoch automatically

In [ ]:
class KerasModelWrapper(BaseEstimator):
    def __init__(self, keras_model):
        self.keras_model = keras_model

    def fit(self, X, y):
        self.keras_model.fit(X, y)

    def predict_proba(self, X):
        return self.keras_model.predict_proba(X)

    def score(self, X, y):
        y_pred = self.keras_model.predict(X)
        if y.shape[1] > 1:
            # Handle multilabel-indicator targets
            y_pred = (y_pred > 0.5).astype(int)
        else:
            # Handle binary targets
            y_pred = np.argmax(y_pred, axis=-1)
        return 1 - hamming_loss(y_true=y, y_pred=y_pred)

# Create an instance of the KerasModelWrapper class
keras_wrapper = KerasModelWrapper(model)

# Fit the PermutationImportance object to the data
perm = PermutationImportance(keras_wrapper, random_state=1).fit(train_X, train_Y)

# Display the feature importances
eli5.show_weights(perm, feature_names=table.columns[:-1].tolist())

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(history.history['loss'], 'b-', label='training loss')
plt.plot(history.history['val_loss'], 'r--', label='test loss')
plt.xlabel('Epoch')
plt.xticks([0, 50, 100, 150, 200, 250], fontsize=13)
plt.yticks(fontsize=13)
plt.ylim(0.0, 1)
plt.legend(fontsize=13)

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], 'g-', label='training accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='test accuracy')
plt.xlabel('Epoch')
plt.ylim(0.0, 1)
plt.xticks([0, 50, 100, 150, 200, 250], fontsize=13)
plt.yticks(fontsize=13)
plt.ylim(0.0, 1)
plt.legend(fontsize=13)

plt.show

In [ ]:
result = model.evaluate(test_X, test_Y)

In [ ]:
pred_Y = model.predict(test_X)
y1=np.delete(pred_Y,0,1)
y2=np.transpose(y1)

In [ ]:
x1=np.delete(test_Y,0,1)
x2=np.transpose(x1)

In [ ]:
df1 = pd.DataFrame(data=x1,columns=['Test'])
df2 = pd.DataFrame(data=y1, columns=['Predict'])
df3 = pd.concat([df1, df2], axis = 1, ignore_index=False)
print("This is a preview of test data prediction results")
print(df3.head())

# Check the stored file to open full results
filename = datetime.datetime.now().strftime("%m%d%H%M%S")

In [ ]:
model.summary()
print("Activation function was",ActFunc)

In [ ]:
# Save the trained model in the *.h5 format
model.save("trained_model_HPC.h5")

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = x1
y_pred = np.round(y1,0)
confusion_matrix(y_true, y_pred)

matrix = confusion_matrix(y_true, y_pred, labels=[0, 1])
print('confusion_matrix : \n', matrix)

In [ ]:
TN = matrix[0,0]
TP = matrix[1,1]
FN = matrix[1,0]
FP = matrix[0,1]

#accuracy = (TP+TN)/(TP+FN+FP+TN)
sens = TP/(TP + FN) 
spec = TN/(TN + FP) 

In [ ]:
# Final results
plt.figure(figsize=(6,6))
sb.stripplot(x="Test", y="Predict", hue="Test", data=df3, linewidth=1, size=13, marker="o", edgecolors="Set1", alpha=0.7)
plt.ylim(-0.1, 1.1)
plt.xlim(-0.3, 1.3)
plt.xticks([0.0,0.5,1.0], fontsize=14)
plt.yticks([0.0,0.5,1.0], fontsize=14)
plt.hlines(0.5,-1,2, color="lightgrey", linestyle='dashed')
plt.vlines(0.5,-1,2, color="lightgrey", linestyle='dashed')
plt.xlabel('True HPC', fontsize=15)
plt.ylabel('Predicted HPC', fontsize=15)
plt.legend(('Pass','Fail'), fontsize=12, loc='center')

plt.text(1.4, 1.0, ('input',table.columns))
plt.text(1.4, 0.9, ('activation function=', ActFunc, '1st nodes=', nodes1))
plt.text(1.4, 0.8, ('Loss,acc=', result))
plt.text(1.4, 0.7, ('sensitivity=', sens))
plt.text(1.4, 0.6, ('specificity=', spec))

# customize your directory 
#os.chdir(r"C:\")
figname = datetime.datetime.now().strftime("%m%d%H%M%S")
plt.savefig("Fig_"+figname+".png", facecolor='white', bbox_inches='tight')
plt.show()
print("This is the final result by HPC pass/fail prediction by culture-independent FCM-ICC data")